In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

from PIL import Image
from pathlib import Path
from numpy import linalg as LA
from numpy.linalg import inv

# Data processing

In [22]:
# chose 25 folders randomly
chosen = np.random.default_rng(2022).choice(np.arange(1, 68 + 1), 25, replace=False)

# Load images 
images = []
labels = []

directory = Path("/Python_programme/Jupyter/PIE_origin")
for i in range(len(chosen)):
    # split train and test
    sub_images = []
    sub_labels = []
    sub_directory = directory / str(chosen[i])
    
    files = list(sub_directory.glob("*.jpg"))
    np.random.default_rng(2023).shuffle(files)
    for filename in files:
        im = Image.open(filename)
        sub_images.append(np.array(im))
        sub_labels.append(i)  # Use folder numbers as labels
    images.append(sub_images)
    labels.append(sub_labels)
    
# len(images)
# len(labels)

In [27]:
# process own photos
own_images = []
own_labels = []

directory = Path("/Python_programme/Jupyter/own_photos")
for filename in directory.glob("*.jpg"):
    im = Image.open(filename)
    own_images.append(np.array(im))
    own_labels.append(25)

In [28]:
# Train-test split
NUM_TRAIN_IMAGES_PER_SUBJECT = np.int_(np.around(0.7 * 170))

images_train, images_test = np.split(np.array(images), [NUM_TRAIN_IMAGES_PER_SUBJECT], axis=1)
labels_train, labels_test = np.split(np.array(labels), [NUM_TRAIN_IMAGES_PER_SUBJECT], axis=1)
own_images_train, own_images_test = np.split(np.array(own_imgs), [7])
own_labels_train, own_labels_test = np.split(np.array(own_labels), [7])

import warnings
warnings.filterwarnings('ignore', category=Warning)

# Flatterning
images_train = images_train.reshape(2975, 32, 32, 1)
own_images_train = own_images_train.reshape(7, 32, 32, 1)
images_test = images_test.reshape(1275, 32, 32, 1)
own_images_test = own_images_test.reshape(3, 32, 32, 1)

labels_train = labels_train.reshape(2975)
labels_test = labels_test.reshape(1275)

# Combine PIE images and own photos to get full train and test sets
full_train_set = np.append(images_train,own_images_train,axis=0,)
full_train_labels = np.append(labels_train, own_labels_train)

full_test_set = np.append(images_test,own_images_test,axis=0,)
full_test_labels = np.append(labels_test, own_labels_test)

# CNN

In [29]:
# Create datasets for tensorflow
train_dataset = tf.data.Dataset.from_tensor_slices((full_train_set, tf.keras.utils.to_categorical(full_train_labels)))
test_dataset = tf.data.Dataset.from_tensor_slices((full_test_set, tf.keras.utils.to_categorical(full_test_labels)))

train_dataset = (train_dataset.cache().shuffle(100).batch(128).prefetch(buffer_size=tf.data.AUTOTUNE))  # choose batch-size of 128
test_dataset = test_dataset.cache().batch(128).prefetch(buffer_size=tf.data.AUTOTUNE)

# Create the model
tf.keras.backend.clear_session()
# 1 channel; maxpool kernal size: 2, stride: 2; conv kernal size:5
model = tf.keras.Sequential(
    [
        tf.keras.layers.Rescaling(1.0 / 255, input_shape=(32, 32, 1)),           
        tf.keras.layers.Conv2D(20, 5, activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2),strides=(2, 2),),
        tf.keras.layers.Conv2D(50, 5, activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2),strides=(2, 2),),
        tf.keras.layers.Flatten(),  
        tf.keras.layers.Dense(500, activation="relu"),
        tf.keras.layers.Dense(26, activation="softmax"),
    ]
)

# optimization
model.compile(optimizer=tf.keras.optimizers.Adam(amsgrad=True), loss=tf.keras.losses.CategoricalCrossentropy(),  metrics=["accuracy"],)

# show the model parameters
model.summary()
# create a callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 20)        520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 20)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 50)        25050     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 50)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1250)              0

In [30]:
# train the model for 100 epochs
model.fit(train_dataset, epochs=100, callbacks=[tensorboard_callback])

Epoch 1/100
24/24 [==============================] - 8s 140ms/step - loss: 3.3272 - accuracy: 0.0104
Epoch 2/100
24/24 [==============================] - 3s 121ms/step - loss: 3.2735 - accuracy: 0.0402
Epoch 3/100
24/24 [==============================] - 3s 119ms/step - loss: 3.2654 - accuracy: 0.0399
Epoch 4/100
24/24 [==============================] - 2s 102ms/step - loss: 3.2613 - accuracy: 0.0399
Epoch 5/100
24/24 [==============================] - 3s 113ms/step - loss: 3.2564 - accuracy: 0.0399
Epoch 6/100
24/24 [==============================] - 3s 136ms/step - loss: 3.2554 - accuracy: 0.0399
Epoch 7/100
24/24 [==============================] - 3s 119ms/step - loss: 3.2543 - accuracy: 0.0399
Epoch 8/100
24/24 [==============================] - 3s 130ms/step - loss: 3.2534 - accuracy: 0.0399
Epoch 9/100
24/24 [==============================] - 3s 126ms/step - loss: 3.2514 - accuracy: 0.0426
Epoch 10/100
24/24 [==============================] - 3s 129ms/step - loss: 3.2530 - accura

24/24 [==============================] - 3s 131ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 82/100
24/24 [==============================] - 3s 140ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 83/100
24/24 [==============================] - 3s 116ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 84/100
24/24 [==============================] - 3s 112ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 85/100
24/24 [==============================] - 3s 135ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 86/100
24/24 [==============================] - 3s 111ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 87/100
24/24 [==============================] - 3s 119ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 88/100
24/24 [==============================] - 3s 120ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 89/100
24/24 [==============================] - 3s 112ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 90/100
24/24 [==============================] - 2s 100ms/step - loss: 0.0033 - accuracy: 

In [31]:
# test the model on testing set
model.predict(test_dataset)

10/10 [==============================] - 0s 28ms/step


array([[9.9999404e-01, 6.6695702e-07, 4.5061617e-14, ..., 3.5094615e-11,
        1.0366610e-07, 0.0000000e+00],
       [9.9995613e-01, 1.1959507e-06, 5.3550246e-12, ..., 6.0982029e-07,
        9.9722419e-10, 1.9179794e-33],
       [9.9998713e-01, 8.0294527e-09, 4.4132418e-16, ..., 4.5453524e-15,
        1.6064615e-11, 0.0000000e+00],
       ...,
       [0.0000000e+00, 5.0942453e-25, 4.1376842e-21, ..., 2.6732343e-20,
        4.2363439e-34, 1.0000000e+00],
       [2.6826154e-30, 5.6072278e-12, 5.2080138e-15, ..., 1.9441885e-12,
        2.1782575e-22, 9.9999869e-01],
       [0.0000000e+00, 0.0000000e+00, 4.6649361e-36, ..., 4.8647062e-34,
        0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [32]:
# compute loss and accuracy
model.evaluate(test_dataset)

10/10 [==============================] - 1s 28ms/step - loss: 0.1392 - accuracy: 0.9695


[0.1392398625612259, 0.9694835543632507]